# SIVA(シヴァ)AI競馬開発エンジニアによる<br>「競馬で始める機械学習」ハンズオン

## 目標
    ・競馬の過去データを使って、予測AIの作成手順を身につける。
    ・前処理、アルゴリズムのチューニングによって予測精度を上げる手法を身につける。
    ・各自作成した競馬予測AIを使って本日開催の競馬の予測を行う。

## 本日の流れ
    1- 必要ライブラリの読み込み
    2- データの読み込み
    3- 前処理
    4- 予測アルゴリズムの構築
    5- 本日の競馬の予測



## 1- 必要ライブラリの読み込み

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from keras.utils.np_utils import to_categorical
from keras.layers import Activation,Dense
from keras.models import Sequential

from keras.layers.normalization import BatchNormalization

import util as ut

## 2- データの読み込み
    2016年~2018年の3年分の過去データ(data_horse_race.csv)を読み込む

In [ ]:
df = pd.read_csv('data/data_horse_race.csv')

In [ ]:
for c in df.columns:
    print(c,' : ',df.iloc[0][c])

## 3- 前処理
    ・不要ファクタの削除
    ・カテゴリカルデータのone-hot値化

### ラベルデータの作成
    今回ラベルは[3着以下,3着以上]と設定する

In [ ]:
#着順データを抽出する
target_data = df['着順']

In [ ]:
target_data[:15]

In [ ]:
#着順がN着以内であれば1,3着以上であれば0に変換

#分類分けの閾値を設定
limit_order = 3

target_data = target_data.apply(lambda x:1 if x <= limit_order else 0)

In [ ]:
target_data[:15]

### 不要カラムを削除

In [ ]:
#カスタマイズ用前処理プログラム
def preprocessing(df):
    #make_yourself!!
    return df

In [ ]:
df = preprocessing(df)

In [ ]:
def drop_columns(df):
    #不要カラムのリストを作成しまとめて削除する
    drop_columns_list = []

    #馬を特定する情報を削除
    drop_columns_list.extend(['血統番号','馬名'])

    #レース前にわからないデータを削除リストに追加
    drop_columns_list.extend(['着順','走破タイム','着差'])

    #カテゴリデータであり種類が多いため、取捨選択をよく考える必要がある
    drop_columns_list.extend(['調教師コード','騎手コード'])
    
    drop_columns_list.extend(['開催年'])

    pre_col = ['騎手名','騎手コード']
    for i in range(1,4):
        for c in pre_col:
            drop_columns_list.append(c + '_'+str(i)+'走前')

    #直前発表データのため今回は不使用
    drop_columns_list.extend(['天気コード','芝馬場','ダート馬場','馬体重','増減符号','増減差'])

    df = df.drop(drop_columns_list,axis = 1)
    
    return df

In [ ]:
df = drop_columns(df)

### カテゴリカル変数をone-hot値に変更する

In [ ]:
def to_onehot(df):
    #カテゴリカル変数のリストを作成しまとめて削除する
    dummies_columns_list = []

    dummies_columns_list.extend(['競馬場コード','性別','毛色コード','東西所属コード',
                            'ブリンカー仕様区分','グレードコード','競争種別コード','重量種別コード',
                            '競争条件コード','芝ダート','回り方向','コース区分'])

    #過去3走分データのカテゴリカル変数
    pre_col = ['競馬場コード', 'ブリンカー仕様区分','異常区分コード','グレードコード'
               ,'競争種別コード','重量種別コード','競争条件コード','芝ダート'
               ,'回り方向','コース区分','天気コード','芝馬場','ダート馬場',]
    for i in range(1,4):
        for c in pre_col:
            dummies_columns_list.append(c + '_'+str(i)+'走前')

    df = pd.get_dummies(data = df,columns = dummies_columns_list)
    
    return df

In [ ]:
df = to_onehot(df)

### 空値をカラムの[0,中央値,平均値]などで埋める

In [ ]:
def fill_nan(df):
    #空値を0で埋める
    df = df.fillna(0)

    #空値を中央値で埋める
    #df = df.fillna(df.median())

    #空値を平均値で埋める
    #df = df.fillna(df.mean())
    
    return df

In [ ]:
df = fill_nan(df)

### 数値を正規化する(ニューラルネットの場合)

In [ ]:
def normalize(df):
    for c in df.columns:
        df[c] = scale(df[c].astype('float'))

    return df

In [ ]:
df = normalize(df)

### 予測用データの作成

In [ ]:
#numpy配列に変換
x = df.values
y = target_data.values

### 訓練とテストにデータを分ける
    分割割合はtest_sizeで指定

In [ ]:
data_train , data_test , target_train , target_test = train_test_split(x,y,test_size=0.2)

## 4- 予測アルゴリズムの構築と精度検証

### ランダムフォレスト(分類)

In [ ]:
forest_classifier = RandomForestClassifier(min_samples_leaf=3, random_state=0)
forest_classifier.fit(data_train, target_train)

In [ ]:
print('Train score: {}'.format(forest_classifier.score(data_train, target_train)))
print('Test score: {}'.format(forest_classifier.score(data_test, target_test)))

In [ ]:
#テストデータの内10件の予測結果
output = forest_classifier.predict(data_test)
for (t,o) in zip(target_test[:10],output[:10]):
    print('target : ',t,'predict : ',o)

### ランダムフォレスト (回帰)

In [ ]:
forest_regressor = RandomForestRegressor(min_samples_leaf=3, random_state=0)
forest_regressor.fit(data_train, target_train)

In [ ]:
print('Train score: {}'.format(forest_regressor.score(data_train, target_train)))
print('Test score: {}'.format(forest_regressor.score(data_test, target_test)))

In [ ]:
#テストデータの内10件の予測結果
output = forest_regressor.predict(data_test)
for (t,o) in zip(target_test[:10],output[:10]):
    print('target : ',t,'predict : ',o)

### ニューラルネット

In [ ]:
model = Sequential()
model.add(Dense(400, input_dim=len(x[0])))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(300))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(200))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
target_train = to_categorical(target_train)
target_test = to_categorical(target_test)

In [ ]:
model.fit(data_train, target_train, batch_size=100, epochs=5, verbose=1,validation_data= (data_test,target_test))

In [ ]:
output = model.predict(data_test)

In [ ]:
#テストデータの内10件の予測結果
for (t,o) in zip(target_test[:10],output[:10]):
    print('target : ',t,'predict : ',o)

## 5- 本日の競馬の予測

In [ ]:
#本日開催の競馬データの読み込み
today_data = pd.read_csv('data/today_data.csv')

In [ ]:
today_data = today_data[(today_data['競馬場コード'] == 6)&(today_data['レース番号'] == 11)]

In [ ]:
#過去データの読み込み
df = pd.read_csv('data/data_horse_race.csv')

In [ ]:
#あとで予測対象データを取り出しやすいようindexを変更する
today_data.index = [i+1000000 for i in range(len(today_data))]

In [ ]:
#前処理のため過去データを連結する
df = pd.concat([df,today_data])

In [ ]:
#先ほど作った前処理関数
df = preprocessing(df)
df = drop_columns(df)
df = to_onehot(df)
df = fill_nan(df)
df = normalize(df)

In [ ]:
#予測対象データを取り出す
df = df.ix[1000000:]

In [ ]:
x = df.values

## 予測結果の出力

### ランダムフォレスト(分類)

In [ ]:
#予測出力
output = forest_classifier.predict(x)

In [ ]:
print(ut.course(today_data.iloc[0]['競馬場コード']),today_data.iloc[0]['レース番号'],'R')
for (t,o) in zip(today_data[['馬番','馬名']].values,output):
    print(str(t[0])+'番\t' + t[1] +'\t' +str(o))

### ランダムフォレスト(回帰)

In [ ]:
#予測出力
output = forest_regressor.predict(x)

In [ ]:
print(ut.course(today_data.iloc[0]['競馬場コード']),today_data.iloc[0]['レース番号'],'R')
for (t,o) in zip(today_data[['馬番','馬名']].values,output):
    print(str(t[0])+'番\t' + t[1] +'\t' +str(o[1]))

### ニューラルネット

In [ ]:
#予測出力
output = model.predict(x)

In [ ]:
print(ut.course(today_data.iloc[0]['競馬場コード']),today_data.iloc[0]['レース番号'],'R')
for (t,o) in zip(today_data[['馬番','馬名']].values,output):
    print(str(t[0])+'番\t' + t[1] +'\t' +str(o[1]))